In [ ]:
import requests
from io import BytesIO
import imageio
import rasterio
from rasterio.io import MemoryFile
from rasterio.plot import show
import matplotlib.pyplot as plt
import numpy as np
import json

In [ ]:
class GeoData:
    HOST = 'http://localhost:8000'
    def __init__(self,):
        pass

    @staticmethod
    def _load_image(content):
        return imageio.imread(BytesIO(content))

    @staticmethod
    def _load_raster(content):
        memfile = MemoryFile(content)
        dataset = memfile.open()
        return dataset

    @staticmethod
    def get_subsampled_raster(pk):
        a = requests.get(f'{GeoData.HOST}/api/geodata/imagery/subsample/{pk}')
        b = requests.get(a.json()['data'])
        #return GeoData._load_image(b.content)
        return GeoData._load_raster(b.content)

    @staticmethod
    def get_image_entry(pk):
        a = requests.get(f'{GeoData.HOST}/api/geodata/imagery/image_entry/{pk}/data')
        #return GeoData._load_image(a.content)
        return GeoData._load_raster(a.content)
    
    @staticmethod
    def post_subsample(payload):
        a = requests.post(f'{GeoData.HOST}/api/geodata/imagery/subsample', json=payload)
        return json.loads(a.content)


## DEM Subsampling

In [ ]:
img_id = 174

In [ ]:
src = GeoData.get_image_entry(img_id)
show(src)

In [ ]:
points = np.array([[-107.08212524738178, 39.01040379702808], 
                   [-106.96182164246767, 39.03110215679572], 
                   [-106.90895466037738, 38.98387516880551], 
                   [-106.9805540376965, 38.91038429753703], 
                   [-107.07130208569401, 38.952157178475225], 
                   [-107.08212524738178, 39.01040379702808]])

shape = {"type": "Polygon", 
         "coordinates": [points.tolist(), ]}

In [ ]:
ax = plt.gca()
show(src, ax=ax)
plt.plot(points[:,0], points[:,1], linewidth=5, color='k')

In [ ]:
payload = {
            'source_image': img_id,
            'sample_type': 'geojson',
            'sample_parameters': shape,
        }

result = GeoData.post_subsample(payload)
result

In [ ]:
product = GeoData.get_subsampled_raster(result['pk'])

In [ ]:
ax = plt.gca()
show(src, ax=ax, cmap='Greys')
show(product, ax=ax)
plt.plot(points[:,0], points[:,1], linewidth=5, color='k')
plt.show()

## Annotation Subsampling

In [ ]:
img_id = 145

In [ ]:
src = GeoData.get_image_entry(img_id)
# show(src)

In [ ]:
payload = {
            'source_image': img_id,
            'sample_type': 'annotation',
            'sample_parameters': {'id': 161, 'outline': True},
        }

result = GeoData.post_subsample(payload)
result

In [ ]:
product = GeoData.get_subsampled_raster(result['pk'])

In [ ]:
ax = plt.gca()
show(src, ax=ax)
show(product, ax=ax, )
ax.set_xlim(0, src.shape[1])
ax.set_ylim(src.shape[0], 0)
plt.show()

In [ ]:
src.shape